In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import os

# Load train.xlsx and val.xlsx
train_df = pd.read_excel('/content/train2.xlsx')
val_df = pd.read_excel('/content/val2.xlsx')

# 경로 수정
train_df['names'] = train_df['names'].str.replace('\\', '/')

train_images = train_df['names'].tolist()
train_labels = train_df['labels'].tolist()

val_images = val_df['names'].tolist()
val_labels = val_df['labels'].tolist()

# class_reduce = 0.1  # 부류 수 줄여서 데이터양 줄임(속도와 메모리 효율을 위해)
no_class = len(train_df['labels'].unique())  # 부류 개수
x_train, y_train = [], []

for image_path, label in zip(train_images, train_labels):
    img = image.load_img(image_path, target_size=(32, 32))
    if len(img.getbands()) != 3:
        print("주의: 유효하지 않은 영상 발생", image_path)
        continue
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x_train.append(x)
    y_train.append(label)

x_test, y_test = [], []

for image_path, label in zip(val_images, val_labels):
    img = image.load_img(image_path, target_size=(32, 32))
    if len(img.getbands()) != 3:
        print("주의: 유효하지 않는 영상 발생", image_path)
        continue
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x_test.append(x)
    y_test.append(label)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
y_train = tf.keras.utils.to_categorical(y_train, no_class)
y_test = tf.keras.utils.to_categorical(y_test, no_class)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))  # ResNet에서 특징 추출 부분 설정
cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))  # 뒷부분을 새로 부착
cnn.add(Dense(no_class, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])  # 미세 조정 방식의 학습(낮은 학습률 설정)
hits = cnn.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test), verbose=1)

res = cnn.evaluate(x_test, y_test, verbose=0)
print("정확도는 ", res[1]*100)


Epoch 1/10
529/529 [==============================] - 813s 1s/step - loss: 1.1113 - accuracy: 0.6355 - val_loss: 1.5429 - val_accuracy: 0.3872
Epoch 2/10
529/529 [==============================] - 780s 1s/step - loss: 1.0123 - accuracy: 0.6267 - val_loss: 1.6620 - val_accuracy: 0.5896
Epoch 3/10
529/529 [==============================] - 789s 1s/step - loss: 0.9020 - accuracy: 0.6650 - val_loss: 9.1608 - val_accuracy: 0.6431
Epoch 4/10
529/529 [==============================] - 787s 1s/step - loss: 0.8696 - accuracy: 0.6861 - val_loss: 1.2669 - val_accuracy: 0.6812
Epoch 5/10
529/529 [==============================] - 788s 1s/step - loss: 1.0654 - accuracy: 0.5878 - val_loss: 1.4197 - val_accuracy: 0.4407
Epoch 6/10
 68/529 [==>...........................] - ETA: 11:08 - loss: 1.0384 - accuracy: 0.5809

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import os

# Load train.xlsx and val.xlsx
train_df = pd.read_excel('/content/train2.xlsx')
val_df = pd.read_excel('/content/val2.xlsx')

# 경로 수정
train_df['names'] = train_df['names'].str.replace('\', '/')

train_images = train_df['names'].tolist()
train_labels = train_df['labels'].tolist()

val_images = val_df['names'].tolist()
val_labels = val_df['labels'].tolist()

class_reduce = 0.1  # 부류 수 줄여서 데이터양 줄임(속도와 메모리 효율을 위해)
no_class = len(train_df['labels'].unique())  # 부류 개수
x_train, y_train = [], []

for image_path, label in zip(train_images, train_labels):
    img = image.load_img(image_path, target_size=(32, 32))
    if len(img.getbands()) != 3:
        print("주의: 유효하지 않은 이미지 발생", image_path)
        continue
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x_train.append(x)
    y_train.append(label)

x_test, y_test = [], []

for image_path, label in zip(val_images, val_labels):
    img = image.load_img(image_path, target_size=(32, 32))
    if len(img.getbands()) != 3:
        print("주의: 유효하지 않는 이미지 발생", image_path)
        continue
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x_test.append(x)
    y_test.append(label)

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)
y_train = tf.keras.utils.to_categorical(y_train, no_class)
y_test = tf.keras.utils.to_categorical(y_test, no_class)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))  # ResNet에서 특징 추출 부분 설정
cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))  # 뒷부분을 새로 부착
cnn.add(Dense(no_class, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])  # 미세 조정 방식의 학습(낮은 학습률 설정)
hits = cnn.fit(x_train, y_train, batch_size=16, epochs=10, validation_data=(x_test, y_test), verbose=1)

res = cnn.evaluate(x_test, y_test, verbose=0)
print("정확도는 ", res[1]*100)

# 모델 저장
cnn.save("/content/saved_model.h5")

# 저장된 모델 불러오기
loaded_model = load_model("/content/saved_model.h5")

# 저장된 모델로 테스트 진행
test_results = loaded_model.evaluate(x_test, y_test, verbose=0)
print("불러온 모델의 정확도는 ", test_results[1]*100)
